In [31]:
import itertools
import os
from pathlib import Path
import pickle
import sys

from arcgis.gis import GIS
from arcgis.features import GeoAccessor
from arcgis.geometry import Geometry
import dask.dataframe
import pandas as pd

sys.path.append('../../src')
from geoai_retail.analysis import get_add_new_closest_dataframe

sys.path.append('../')
import config

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
data = Path(os.path.abspath('../../data'))
interim_dir = data/'interim'
interim_gdb = interim_dir/'interim.gdb'
raw_dir = data/'raw'
raw_gdb = raw_dir/'raw.gdb'

origin_fc = str(raw_gdb/'sea_block_group')
origin_id_fld = 'ID'

dest_fc = str(raw_gdb/'sea_ace')
dest_id_fld = 'LOCNUM'

comp_fc = raw_gdb/'sea_ace_comp'
comp_id_fld = 'LOCNUM'

real_estate_fc = raw_gdb/'real_estate_hex'
real_estate_id_fld = 'GRID_ID'

closest_brand = interim_dir/'closest_store.csv'
closest_comp = interim_dir/'closest_competition.csv'
origin_demographics = interim_dir/'origin_enrich_all.csv'
inrix_trips = interim_dir/'cust_count_inrix.csv'
training_data = interim_dir/'training_data.csv'

In [4]:
origin_df = GeoAccessor.from_featureclass(str(origin_fc))
origin_df.head()

,OBJECTID,ID,NAME,SHAPE
0,1,530530701003,530530701.003,"{""rings"": [[[-122.06627999999995, 47.076520000..."
1,2,530530714071,530530714.071,"{""rings"": [[[-122.34031999999996, 47.071510000..."
2,3,530530714072,530530714.072,"{""rings"": [[[-122.35767999999996, 47.067370000..."
3,4,530530714073,530530714.073,"{""rings"": [[[-122.36847999999998, 47.067630000..."
4,5,530530714112,530530714.112,"{""rings"": [[[-122.41108999999994, 47.071690000..."


In [5]:
dest_df = GeoAccessor.from_featureclass(str(dest_fc)).set_index('OBJECTID', drop=True)
dest_df.head()

,LOCNUM,CONAME,STREET,CITY,STATE,STATE_NAME,ZIP,ZIP4,NAICS,SIC,...,ISCODE,SQFTCODE,LOC_NAME,STATUS,SCORE,SOURCE,REMOVE,NEAR_FID,NEAR_DIST,SHAPE
OBJECTID,,,,,,,,,,,,,,,,,,,,,
1,371889957,GRAHAM ACE HARDWARE,224TH ST E,GRAHAM,WA,Washington,98338,5704,44413005,525104,...,,6,PointAddress,M,100.0000,INFOGROUP,0,3,0.000021,"{""x"": -122.29909999999995, ""y"": 47.05410000000..."
2,460556608,OAKBROOK ACE HARDWARE,STEILACOOM BLVD SW,TACOMA,WA,Washington,98498,6154,44413005,525104,...,,3,StreetAddress,M,96.3594,INFOGROUP,0,35,0.000010,"{""x"": -122.54489999999998, ""y"": 47.18000000000..."
3,405129289,GIG HARBOR ACE HARDWARE,POINT FOSDICK DR NW,GIG HARBOR,WA,Washington,98335,1711,44413005,525104,...,,7,StreetAddress,M,100.0000,INFOGROUP,0,41,0.000019,"{""x"": -122.57929999999999, ""y"": 47.30130000000..."
4,216082099,SOUTH END ACE HARDWARE,PACIFIC AVE S,SPANAWAY,WA,Washington,98387,8395,44413005,525104,...,,5,PointAddress,M,100.0000,INFOGROUP,0,96,55.586255,"{""x"": -122.43459999999999, ""y"": 47.08740000000..."
5,721714069,ACE HARDWARE,112TH ST S,PARKLAND,WA,Washington,98444,5711,44413005,525104,...,,3,PointAddress,M,95.4063,INFOGROUP,0,98,0.000010,"{""x"": -122.43199999999996, ""y"": 47.15610000000..."


In [6]:
real_estate_df = GeoAccessor.from_featureclass(str(real_estate_fc), where_clause="road_reachable = 1")
real_estate_df = real_estate_df[[real_estate_id_fld, 'SHAPE']].copy()
print(real_estate_df.GRID_ID.dtype)
print(len(real_estate_df.index))
real_estate_df.head()

object
1906


,GRID_ID,SHAPE
0,28,"{""rings"": [[[-121.98783159799996, 46.740620218..."
1,30,"{""rings"": [[[-121.92611773799996, 46.740620218..."
2,31,"{""rings"": [[[-121.89526080699994, 46.728410000..."
3,32,"{""rings"": [[[-121.86440387799996, 46.740620218..."
4,33,"{""rings"": [[[-121.83354694699995, 46.728410000..."


In [7]:
row = real_estate_df.iloc[0]
row

GRID_ID                                                   28
SHAPE      {'rings': [[[-121.98783159799996, 46.740620218...
Name: 0, dtype: object

In [61]:
def _get_rebalanced_closest_df(x_coord, y_coord, new_location_id):
    
    cnt_geom = Geometry({'x': x_coord, 'y': y_coord, 'spatialReference': real_estate_df.spatial.sr})
    
    # get a dataframe of all potential rebalanced areas
    rebal_df = get_add_new_closest_dataframe(
        origins=origin_df,
        origin_id_field=origin_id_fld,
        destinations=dest_df,
        destination_id_field=dest_id_fld,
        closest_table=closest_brand,
        new_destination=cnt_geom
    )
    
    # while we soved way broader than possilby needed, we check and only keep changed rows
    dest_id_cols = [col for col in rebal_df.columns if col.startswith('destination_id')]
    dest_id_arr = dest_df[dest_id_fld].unique()
    new_rebal_origin_ids = pd.concat([rebal_df[~rebal_df[dest_col].isin(dest_id_arr)]['origin_id'] for dest_col in dest_id_cols]).unique()
    rebal_keep_df = rebal_df[rebal_df['origin_id'].isin(new_rebal_origin_ids)].copy()

    rebal_keep_df['new_destination_id'] = new_location_id
    
    return rebal_keep_df

def get_affected_df(input_df):
    
    df_lst = input_df.apply(lambda row: _get_rebalanced_closest_df(
        row['SHAPE'].centroid[0], row['SHAPE'].centroid[1], row[real_estate_id_fld]), axis=1)
    return pd.concat(test_df.values)

In [57]:
test_df = get_affected_df(real_estate_df.iloc[500:502], real_estate_id_fld)
test_df

500            origin_id destination_id_01  proximity...
501            origin_id destination_id_01  proximity...
dtype: object

In [60]:
pd.concat(test_df.values)

,origin_id,destination_id_01,proximity_traveltime_01,proximity_kilometers_01,destination_id_02,proximity_traveltime_02,proximity_kilometers_02,destination_id_03,proximity_traveltime_03,proximity_kilometers_03,destination_id_04,proximity_traveltime_04,proximity_kilometers_04,destination_id_05,proximity_traveltime_05,proximity_kilometers_05,destination_id_06,proximity_traveltime_06,proximity_kilometers_06,new_destination_id
36,530530726011,405129289,21.491815,20.246174,1,41.405133,34.216766,715565781,55.025679,38.879325,715571320,55.025679,38.879325,460556608,39.279178,38.897734,721714069,44.577537,44.321717,1288
37,530530726012,405129289,22.204510,25.707658,1,42.117827,39.678250,715565781,55.248502,44.155208,715571320,55.248502,44.155208,460556608,39.991873,44.359218,721714069,45.290231,49.783201,1288
38,530530726013,405129289,23.694928,23.556213,1,43.608245,37.526806,460556608,41.482291,42.207773,715565781,59.289798,43.924768,715571320,59.289798,43.924768,721714069,46.780649,47.631757,1288
39,530530726022,405129289,25.834003,27.852050,1,45.747321,41.822642,460556608,43.621366,46.503610,715565781,59.554341,48.018616,715571320,59.554341,48.018616,721714069,48.919725,51.927593,1288
40,530530726023,405129289,28.672600,31.549415,1,48.585918,45.520007,460556608,46.459963,50.200974,715565781,64.267470,51.917969,715571320,64.267470,51.917969,721714069,51.758322,55.624958,1288
41,530530726031,405129289,34.262404,37.344800,1,54.175722,51.315393,460556608,52.049767,55.996360,715565781,69.857274,57.713355,715571320,69.857274,57.713355,721714069,57.348126,61.420344,1288
42,530530726032,405129289,36.339953,40.248020,1,56.253271,54.218612,460556608,54.127316,58.899579,715565781,71.934823,60.616574,715571320,71.934823,60.616574,721714069,59.425675,64.323563,1288
43,530530726034,405129289,36.938191,37.626232,1,56.851508,51.596825,460556608,54.725554,56.277792,715565781,72.533061,57.994787,715571320,72.533061,57.994787,721714069,60.023912,61.701776,1288
436,530530724051,405129289,21.264023,13.013283,1,20.816179,15.925675,460556608,37.440476,30.155365,721714069,46.676557,36.710388,688834225,41.565639,41.411130,677129595,44.117399,41.735620,1288
437,530530724052,405129289,12.159949,8.848938,1,13.839080,10.537612,460556608,33.026529,25.590927,721714069,42.262611,32.145950,688834225,37.151692,36.846692,677129595,39.703453,37.171182,1288


In [ ]:
ent_gis = GIS(config.ent_url, username=config.ent_user, password=config.ent_pass)
ent_gis

In [ ]:
import warnings
warnings.filterwarnings('ignore')

real_estate_df.reset_index(inplace=True, drop=True)

for idx, row in real_estate_df.iterrows():
    
    rebal_result = get_affected_df(get_new_tuple(row, real_estate_id_fld))
    
    if idx == 0 and not affected_df:
        affected_df = rebal_result
    else:
        affected_df = affected_df.append(rebal_result)
    print(f'success - {row[0]}')
    
    affected_df.to_csv(affected_csv)

In [44]:
processing_df = pd.DataFrame(columns=['x_coord', 'y_coord', 'new_location_id'])
processing_df.x_coord = real_estate_df.SHAPE.apply(lambda geom: geom.centroid[0])
processing_df.y_coord = real_estate_df.SHAPE.apply(lambda geom: geom.centroid[1])
processing_df.new_location_id = real_estate_df[real_estate_id_fld]
processing_df.head()

,x_coord,y_coord,new_location_id
0,-122.008403,46.740619,28
1,-121.946689,46.740619,30
2,-121.915832,46.728409,31
3,-121.884975,46.740619,32
4,-121.854118,46.728409,33


In [45]:
ddf = dask.dataframe.from_pandas(processing_df.iloc[500:502], chunksize=30)

In [49]:
ddf_lst = ddf.apply(lambda row: _get_rebalanced_closest_df(row[0], row[1], row[2]), axis=1)
ddf_lst

ValueError: Metadata inference failed in `apply`.

You have supplied a custom function and Dask is unable to 
determine the type of output that that function returns. 

To resolve this please provide a meta= keyword.
The docstring of the Dask function you ran should have more information.

Original error is below:
------------------------
ExecuteError('ERROR 030024: Solve returned a failure.\nInsufficient number of valid locations in "Facilities" or "Incidents".\nFailed to execute (Solve).\n', 'occurred at index 0')

Traceback:
---------
  File "C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgis\lib\site-packages\dask\dataframe\utils.py", line 170, in raise_on_meta_error
    yield
  File "C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgis\lib\site-packages\dask\dataframe\core.py", line 4729, in _emulate
    return func(*_extract_meta(args, True), **_extract_meta(kwargs, True))
  File "C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgis\lib\site-packages\dask\utils.py", line 857, in __ca